In [74]:
import imblearn
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import BaggingClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, log_loss, roc_auc_score, RocCurveDisplay, roc_curve
import numpy as np
import pandas as pd
#from IMPJ import DataProcessor
import random
import matplotlib.pyplot as plt
from os.path import basename, exists
import matplotlib as mpl
import random
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.multiclass import OneVsRestClassifier
from tqdm import tqdm
import os

#dp = DataProcessor()

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)

download('https://github.com/AllenDowney/AstronomicalData/raw/main/' + 'az-paper-twocol.mplstyle')
plt.style.use('./az-paper-twocol.mplstyle')

In [75]:
test_size = 0.3

filepath = "..\\Fermi-LAT Data\\fl_numericalonly_nopositional_withclasses.csv"
data = pd.read_csv(filepath, index_col=0)
featurestokeep = [0,1,3,7,10,18,41,42,66,75,76,81,83,87]
rdata=data.iloc[:,featurestokeep]
rdata['CLASS1']=data['CLASS1']
rdata=rdata.dropna()
Y = np.array(rdata['CLASS1'])
X = np.array(rdata.drop(labels='CLASS1',axis=1))
#X,Y = dp.choose_2_vars('CLASS1',data,1,2)

C:\Users\ioanp\AppData\Local\Temp\ipykernel_28016\503294698.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rdata['CLASS1']=data['CLASS1']


In [76]:
Y[Y==2]=1 # non radio galaxies
Y[Y==3]=0 # radio galaxies

init_ratio = len(Y[Y==0])/len(Y[Y==1])

sm = SMOTE(random_state=42, sampling_strategy=init_ratio*5)
ru = RandomUnderSampler(random_state=42, sampling_strategy=1)

scaler = StandardScaler()

print(len(Y[Y==0]))


39


In [77]:
NN = MLPClassifier(activation='tanh',hidden_layer_sizes=(50,50),learning_rate='constant',solver='sgd',
                   max_iter=5000,random_state=5)
BNN = BaggingClassifier(estimator=NN,n_estimators=5,bootstrap=True,verbose=0,n_jobs=-1)
#ovr = OneVsRestClassifier(estimator=BNN, n_jobs=-1,verbose=10)
iteration=0
kf = KFold(n_splits=10)

y_pred=pd.DataFrame()
y_proba=pd.DataFrame()
y_test_df=pd.DataFrame(index=np.arange(0,np.ceil(len(X)/10)))

for train_indices, test_indices in tqdm(kf.split(X)):
    iteration+=1
    x_train=X[train_indices]
    y_train = Y[train_indices]
    x_test=X[test_indices]
    y_test=Y[test_indices]
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)
    x_test = scaler.transform(x_test)
    print(np.shape(y_train))
    #print(f'Initial Ratio: {len(y_train[y_train==0])/len(y_train[y_train==1])}')
    #print(f'Original Dataset Shape: {Counter(y_train)}')
    x_train_sm, y_train_sm = sm.fit_resample(x_train, y_train)
    x_train_res, y_train_res = ru.fit_resample(x_train_sm,y_train_sm)
    #print(f'New Ratio: {len(y_train_res[y_train_res==0])/len(y_train_res[y_train_res==1])}')
    #print(f'Resampled dataset shape {Counter(y_train_res)}')
    BNN.fit(x_train_res, y_train_res)
    y_pred_vals=BNN.predict(x_test)
    y_proba_vals=BNN.predict_proba(x_test)

    if len(y_test)==len(y_test_df):
        pass
    else:
        y_test=np.append(y_test,np.nan)
        y_pred_vals=np.append(y_pred_vals,np.nan)
        y_proba_vals=np.append(y_proba_vals,[np.nan,np.nan])

    y_test_df['iteration '+str(iteration)] = y_test    
    y_pred['iteration '+str(iteration)] = y_pred_vals
    y_proba['class' + str(0) + ' iteration '+ str(iteration)] = y_proba_vals.T[0]
    y_proba['class'+str(1)+' iteration '+str(iteration)]= y_proba_vals.T[1]

outdir = '..\\Direct_Classifier\\wrapper_results\\'+str(len(featurestokeep))+'_features'
if not os.path.exists(outdir):
    os.mkdir(outdir)
y_pred.to_csv('..\\Direct_Classifier\\wrapper_results\\'+str(len(featurestokeep))+'_features\\y_pred.csv')
y_proba.to_csv('..\\Direct_Classifier\\wrapper_results\\'+str(len(featurestokeep))+'_features\\y_proba.csv')
y_test_df.to_csv('..\\Direct_Classifier\\wrapper_results\\'+str(len(featurestokeep))+'_features\\y_test.csv')


0it [00:00, ?it/s]

(1743,)


1it [00:18, 18.70s/it]

(1743,)


2it [00:25, 11.71s/it]

...

8it [01:24,  9.92s/it]

(1744,)


9it [01:33,  9.79s/it]

(1744,)


10it [01:46, 10.60s/it]


In [78]:
print(np.unique(y_train))

''' SCORING METRICS '''

accuracy = accuracy_score(y_test, y_pred)*100
confusion = confusion_matrix(y_test, y_pred)
roc_auc = roc_auc_score(y_test,y_pred)
roc_auc_weightedavg = roc_auc_score(y_test,y_pred,average='weighted')
logloss = log_loss(y_test,y_pred)
f1 = f1_score(y_test,y_pred)
fpr, tpr, thresholds = roc_curve(y_test,y_proba.T[1])

print('The Neural Network accuracy is ' + str(accuracy))
print('The Neural Network ROC AUC Scores are: '+str(roc_auc))
print("The Neural Network's Weighted Average ROC AUC Score is: " + str(roc_auc_weightedavg))
print("The Neural Network's Logarithmic Loss Score is: " + str(logloss))
print('The Neural Network F1 Score is: '+str(f1))
print('The Neural Network Confusion Matrix is:')
print(confusion)

resultcols = ['Accuracy', 'ROC AUC', 'ROC Weighted Av', 'Logarithmic Loss',
            'F1 Score', 'CMatrix11','CMatrix12', 'CMatrix21', 'CMatrix22']
resultarray = np.array((accuracy,roc_auc,roc_auc_weightedavg,logloss,f1,
                        confusion[0,0],confusion[0,1],confusion[1,0],confusion[1,1]))
rocresultcols = ['FPR','TPR','Thresholds']
rocresultarray = np.array([fpr,tpr,thresholds])
#print(rocresultarray)
results = pd.DataFrame([resultarray],columns=resultcols)
rocresults = pd.DataFrame(np.array([fpr,tpr,thresholds]).transpose(),columns=rocresultcols)

#results.to_csv("/content/drive/My Drive/Colab Notebooks/imbalanced_trials/results1.csv")
#rocresults.to_csv("/content/drive/My Drive/Colab Notebooks/imbalanced_trials/rocresults1.csv")


fig, ax = plt.subplots(figsize=(5,5))

ax.plot(fpr,tpr,linestyle='dashed',marker='x')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

C:\Users\ioanp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\_array_api.py:245: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


[0 1]


ValueError: Input y_true contains NaN.